In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait # 화면이 나올때까지 대기
from selenium.webdriver.support import expected_conditions as EC    # 화면상태체크
import requests
from urllib.parse import quote
from bs4 import BeautifulSoup
import time
import os
import csv
import re
from geopy.geocoders import Nominatim

In [3]:
url = "https://www.bluer.co.kr/search?query=&foodTypeDetail=137&zone1=%EC%84%9C%EC%9A%B8%20%EA%B0%95%EB%82%A8&food2=%ED%8C%8C%EC%8A%A4%ED%83%80"    # 파스타

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Referer": "www.bluer.co.kr"
}
browser = webdriver.Chrome()    # 창열기
browser.maximize_window() # 최대창으로 확대
browser.get(url)

In [3]:
# === 라이브러리 및 폴더 설정 ===
# url = "https://www.bluer.co.kr/search?query=&foodTypeDetail=137&zone1=%EC%84%9C%EC%9A%B8%20%EA%B0%95%EB%82%A8&food2=%ED%8C%8C%EC%8A%A4%ED%83%80"    # 파스타
location_qs = [
    {"locno": 1, "loc": "서울 강남", "is_main": "n", "sort": 1},
    {"locno": 2, "loc": "서울 강북", "is_main": "n", "sort": 2},
]
locationDetail_qs = [
    {"locdno": 1, "location": location_qs[0], "locd_nm": "가로수길/신사역", "is_main": "y", "sort": 1},
    {"locdno": 2, "location": location_qs[0], "locd_nm": "압구정동/도산공원", "is_main": "y", "sort": 2},
    {"locdno": 3, "location": location_qs[0], "locd_nm": "청담동/강남구청역", "is_main": "y", "sort": 3},
    {"locdno": 4, "location": location_qs[0], "locd_nm": "논현동/영동시장", "is_main": "y", "sort": 4},
    {"locdno": 5, "location": location_qs[0], "locd_nm": "삼성동/대치동", "is_main": "y", "sort": 5},
    {"locdno": 6, "location": location_qs[0], "locd_nm": "선릉역/선정릉역", "is_main": "y", "sort": 6},
    {"locdno": 7, "location": location_qs[0], "locd_nm": "역삼동", "is_main": "y", "sort": 7},
    {"locdno": 8, "location": location_qs[0], "locd_nm": "도곡/양재/매봉", "is_main": "y", "sort": 8},
    {"locdno": 9, "location": location_qs[0], "locd_nm": "강남역", "is_main": "y", "sort": 9},
    {"locdno": 10, "location": location_qs[0], "locd_nm": "고속터미널/반포", "is_main": "y", "sort": 10},
    {"locdno": 11, "location": location_qs[0], "locd_nm": "교대/남부터미널/서초", "is_main": "y", "sort": 11},
    {"locdno": 12, "location": location_qs[0], "locd_nm": "방배/사당/이수", "is_main": "y", "sort": 12},
    {"locdno": 13, "location": location_qs[0], "locd_nm": "서래마을", "is_main": "y", "sort": 13},
    {"locdno": 14, "location": location_qs[0], "locd_nm": "서울대/관악구", "is_main": "y", "sort": 14},
    {"locdno": 15, "location": location_qs[0], "locd_nm": "가산디지털단지/구로", "is_main": "y", "sort": 15},
    {"locdno": 16, "location": location_qs[0], "locd_nm": "영등포", "is_main": "y", "sort": 16},
    {"locdno": 17, "location": location_qs[0], "locd_nm": "여의도", "is_main": "y", "sort": 17},
    {"locdno": 18, "location": location_qs[0], "locd_nm": "김포/마곡", "is_main": "y", "sort": 18},
    {"locdno": 19, "location": location_qs[0], "locd_nm": "목동/화곡", "is_main": "y", "sort": 19},
    {"locdno": 20, "location": location_qs[0], "locd_nm": "개포/일원/수서", "is_main": "y", "sort": 20},
    {"locdno": 21, "location": location_qs[0], "locd_nm": "가락/문정/오금/위례", "is_main": "y", "sort": 21},
    {"locdno": 22, "location": location_qs[0], "locd_nm": "잠실/방이/올림픽공원", "is_main": "y", "sort": 22},
    {"locdno": 23, "location": location_qs[0], "locd_nm": "석촌호수", "is_main": "y", "sort": 23},
    {"locdno": 24, "location": location_qs[0], "locd_nm": "강동구", "is_main": "y", "sort": 24},
    {"locdno": 25, "location": location_qs[1], "locd_nm": "노원/강북", "is_main": "y", "sort": 25},
    {"locdno": 26, "location": location_qs[1], "locd_nm": "동대문/성동", "is_main": "y", "sort": 26},
    {"locdno": 27, "location": location_qs[1], "locd_nm": "종로/중구", "is_main": "y", "sort": 27},
]
# 폴더 생성
os.makedirs("output1", exist_ok=True)
os.makedirs("images", exist_ok=True)

# CSV 파일 경로
restaurants_csv = "output1/restaurants.csv"
times_csv = "output1/operating_times.csv"
images_csv = "output1/images.csv"
menus_csv = "output1/menus.csv"

headers = {"User-Agent": "Mozilla/5.0"}

# CSV 헤더 작성
csv_headers = [
    (restaurants_csv, ["resno","location_id","locationDetail_id","res_nm","addr","desc","lat","lng","tel"]),
    (times_csv, ["resno","location_id","locationDetail_id","opno","week","open_time","close_time","desc"]),
    (images_csv, ["resno","location_id","locationDetail_id","imgno","img_url","img_nm"]),
    (menus_csv, ["resno","location_id","locationDetail_id","foodType_id","fno","fnm","price"])
]
for file, header in csv_headers:
    with open(file, "w", newline="", encoding="utf-8-sig") as f:
        csv.writer(f).writerow(header)
# 시작 resno
resno_index = 1
fno_index = 1
opno_index = 1
imgno_index = 1

location_id = 1
locationDetail_id = 1
foodType_id = 1
for locd in locationDetail_qs:
    zone1 = locd["location"]["loc"]   # ex: 서울 강남
    zone2 = locd["locd_nm"]           # ex: 가로수길/신사역
    locationDetail_id = int(locd["locdno"])
    
    url = f"https://www.bluer.co.kr/search?query=&zone1={quote(zone1)}&zone2={quote(zone2)}&foodTypeDetail=140"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
        "Referer": "www.bluer.co.kr"
    }
    browser = webdriver.Chrome()    # 창열기
    browser.maximize_window() # 최대창으로 확대
    browser.get(url)

    wait = WebDriverWait(browser, 10)

    # 지오코더 생성 (User-Agent 꼭 필요)
    # =====================
    # Google Maps에서 이름+주소로 위도, 경도 가져오기
    # =====================
    def get_lat_lng_google_tab(browser, name, address, wait_time=8):
        """
        Google Maps에서 식당 이름+주소로 위도, 경도 가져오기
        browser    : Selenium Chrome WebDriver
        name       : 식당 이름
        address    : 식당 주소
        wait_time  : 검색 후 대기 시간 (초)
        """


        lat, lng = None, None
        
        def search_maps(query):
            # 새 탭 열기
            original_handle = browser.current_window_handle
            browser.execute_script("window.open('about:blank');")
            browser.switch_to.window(browser.window_handles[-1])
            browser.get("https://www.google.com/maps")
            try:
                search_box = WebDriverWait(browser, 10).until(
                    EC.presence_of_element_located((By.ID, "searchboxinput"))
                )
                search_box.clear()
                search_box.send_keys(query)
                search_box.send_keys(Keys.ENTER)
                WebDriverWait(browser, wait_time).until(
                    lambda b: re.search(r'(@-?\d+\.\d+,-?\d+\.\d+|!3d-?\d+\.\d+!4d-?\d+\.\d+)', b.current_url)
                )
                time.sleep(1)
                current_url = browser.current_url
                # !3d37.xxx!4d127.xxx 패턴 우선
                m = re.search(r'!3d(-?\d+\.\d+)!4d(-?\d+\.\d+)', current_url)
                if m:
                    return map(float, m.groups())
                # @37.xxx,127.xxx 패턴
                m2 = re.search(r'@(-?\d+\.\d+),(-?\d+\.\d+)', current_url)
                if m2:
                    return map(float, m2.groups())
            except:
                return None, None
            finally:
                # 새 탭 닫기
                browser.close()
                browser.switch_to.window(original_handle)
            return None, None

        # 1️⃣ 이름 + 주소
        lat, lng = search_maps(f"{name} {address}")
        
        # 2️⃣ 실패하면 이름만
        if lat is None or lng is None:
            print(f"주소 검색 실패, 이름만 시도: {name}")
            lat, lng = search_maps(name)
        return lat, lng

    # 요일 함수
    DAY_ORDER = ["월","화","수","목","금","토","일"]
    def expand_days(day_text):
        days = []
        parts = [p.strip() for p in day_text.split(",")]

        for part in parts:
            part = part.replace("요일", "")

            if "~" in part:
                start, end = part.split("~")
                s_idx = DAY_ORDER.index(start)
                e_idx = DAY_ORDER.index(end)
                if s_idx <= e_idx:
                    days.extend(DAY_ORDER[s_idx:e_idx+1])
                else:
                    days.extend(DAY_ORDER[s_idx:] + DAY_ORDER[:e_idx+1])
            else:
                days.append(part)

        return list(dict.fromkeys(days))  # 중복 제거
    time.sleep(5)

    # === 크롤링 시작 ===
    # =====================
    # 페이지 반복
    # =====================
    while True:
        wait.until(EC.presence_of_element_located((By.ID, "list-restaurant")))
        soup = BeautifulSoup(browser.page_source, "lxml")

        lis = soup.select("#list-restaurant li")
        if not lis:
            print("식당 목록 없음")
            break

        for li in lis:
            title_tag = li.select_one("h3")
            if not title_tag:
                continue
            title = li.select_one("h3").get_text(strip=True)
            address = li.select_one(".juso-info").get_text(strip=True)
            food_types = [f.get_text(strip=True) for f in li.select(".foodtype li")]
            food_type_str = ", ".join(food_types)

            thumb = li.select_one(".thumb-restaurant")
            detail_url = "https://www.bluer.co.kr" + thumb["data-href"]

            tel = ""

            # =====================
            # 상세페이지 (새 탭)
            # =====================
            browser.execute_script("window.open(arguments[0]);", detail_url)
            browser.switch_to.window(browser.window_handles[-1])
            time.sleep(1)

            detail_soup = BeautifulSoup(browser.page_source, "lxml")

            # 전화번호
            tel_tag = detail_soup.select_one("img[src*='icon_phone'] + a")
            if tel_tag:
                tel = tel_tag.get_text(strip=True)

            # 운영시간
            time_div = detail_soup.select_one(".time-info .extra.on")

            if time_div:
                raw_text = time_div.get_text(" ", strip=True)
                desc_text = raw_text  # desc에는 전체 원문 그대로

                # 브레이크타임 제거 (파싱용)
                cleaned = re.sub(r"\(.*?브레이크타임.*?\)", "", raw_text)

                # 🔥 요일 + 시간 패턴 (요일 단어 포함)
                pattern = re.compile(
                    r"([월화수목금토일요일~, ]+)\s+(\d{1,2}:\d{2})~(\d{1,2}:\d{2})"
                )

                for m in pattern.finditer(cleaned):
                    day_text, open_t, close_t = m.groups()

                    # 요일 확장
                    days = expand_days(day_text)

                    for day in days:
                        with open(times_csv, "a", newline="", encoding="utf-8-sig") as f:
                            csv.writer(f).writerow([
                                resno_index,
                                location_id,
                                locationDetail_id,
                                opno_index,
                                day,
                                open_t,
                                close_t,
                                desc_text
                            ])
                        opno_index+=1
            # 이미지
            # for img in detail_soup.select(".owl-stage img"):
            #     img_url = img.get("src")
            #     if img_url and img_url.startswith("/"):
            #         img_url = "https://www.bluer.co.kr" + img_url

            #     img_nm = f"{resno_index}_{img_url.split('/')[-1]}"
            #     save_path = os.path.join("images", img_nm)

            #     try:
            #         r = requests.get(img_url, headers=headers)
            #         if r.status_code == 200:
            #             with open(save_path, "wb") as f:
            #                 f.write(r.content)
            #             with open(images_csv, "a", newline="", encoding="utf-8-sig") as f:
            #                 csv.writer(f).writerow([resno_index, location_id,locationDetail_id,imgno_index,save_path, img_nm])
            #         imgno_index+=1
            #     except:
            #         pass

            # 메뉴
            menu_items = detail_soup.select(".restaurant-menu-list .item-menu")
            seen = set()  # 이미 저장한 메뉴를 기록

            for menu in menu_items:
                name_tag = menu.select_one(".title")
                price_tag = menu.select_one(".price")
                if not (name_tag and price_tag):
                    continue

                name_text = name_tag.get_text(strip=True)
                price_text = price_tag.get_text(strip=True)

                # '원' 바로 앞 숫자만
                price_match = re.search(r'(\d[\d,]*)\s*원', price_text)
                price_value = int(price_match.group(1).replace(",", "")) if price_match else 0

                # 이미 저장한 메뉴면 skip
                if (name_text, price_value) in seen:
                    continue
                seen.add((name_text, price_value))
                
                # 한 번만 저장
                with open(menus_csv, "a", newline="", encoding="utf-8-sig") as f:
                    csv.writer(f).writerow([resno_index, location_id, locationDetail_id, foodType_id, fno_index, name_text, price_value])
                
                fno_index+=1
                    
                # 상세페이지 리뷰
                desc_tag = detail_soup.select_one("div.restaurant-review-info .content")
                if desc_tag:
                    desc_text = desc_tag.get_text(strip=True)
                    # 맨 앞, 맨 뒤 공백 제거 후 양쪽 따옴표 제거
                    food_type_str = desc_text.strip().strip('"')

            browser.close()
            browser.switch_to.window(browser.window_handles[0])

            # 주소에서 위도, 경도 가져오기
            # clean_addr = re.sub(r"\(.*?\)", "", address).strip()
            lat, lng = get_lat_lng_google_tab(browser, title, address)
            # time.sleep(8)  # Nominatim은 요청 제한이 있으므로 3초 쉬는 것이 안전
            
            # 기본 정보 저장
            with open(restaurants_csv, "a", newline="", encoding="utf-8-sig") as f:
                csv.writer(f).writerow([
                    resno_index, location_id, locationDetail_id, title, address, food_type_str, lat, lng, tel
                ])

            print(f"{resno_index} 저장 완료 - {title}")
            resno_index += 1

        # =====================
        # 다음 페이지
        # =====================
        try:
            next_btn = browser.find_element(
                By.CSS_SELECTOR,
                "#page-selection ul li.next:not(.disabled) a"
            )
            next_btn.click()
            time.sleep(2)
        except:
            print("마지막 페이지")
            break

    browser.quit()
    print("✅ 모든 크롤링 완료")


1 저장 완료 - 라모따
마지막 페이지
✅ 모든 크롤링 완료
주소 검색 실패, 이름만 시도: h450
2 저장 완료 - h450
마지막 페이지
✅ 모든 크롤링 완료
3 저장 완료 - 뻬를라
마지막 페이지
✅ 모든 크롤링 완료
주소 검색 실패, 이름만 시도: 콘체르토
4 저장 완료 - 콘체르토
주소 검색 실패, 이름만 시도: 명동피자(신논현점)
5 저장 완료 - 명동피자(신논현점)
주소 검색 실패, 이름만 시도: 도셰프
6 저장 완료 - 도셰프
마지막 페이지
✅ 모든 크롤링 완료
7 저장 완료 - 마노디셰프(삼성점)
8 저장 완료 - 피자느반
9 저장 완료 - 까발로비안코
마지막 페이지
✅ 모든 크롤링 완료
10 저장 완료 - 살루쪼
마지막 페이지
✅ 모든 크롤링 완료
11 저장 완료 - 지아니스나폴리
주소 검색 실패, 이름만 시도: 더키친일뽀르노(역삼센터필드점)
12 저장 완료 - 더키친일뽀르노(역삼센터필드점)
주소 검색 실패, 이름만 시도: 마노디셰프(강남점)
13 저장 완료 - 마노디셰프(강남점)
마지막 페이지
✅ 모든 크롤링 완료
14 저장 완료 - 브리즈
15 저장 완료 - 팔라피자
마지막 페이지
✅ 모든 크롤링 완료
16 저장 완료 - 이엘포레스트
17 저장 완료 - 도치피자(강남점)
마지막 페이지
✅ 모든 크롤링 완료
마지막 페이지
✅ 모든 크롤링 완료
주소 검색 실패, 이름만 시도: 라칼라스
18 저장 완료 - 라칼라스
마지막 페이지
✅ 모든 크롤링 완료
마지막 페이지
✅ 모든 크롤링 완료
19 저장 완료 - 톰볼라
20 저장 완료 - 볼라레
마지막 페이지
✅ 모든 크롤링 완료
21 저장 완료 - 피자파쪼
마지막 페이지
✅ 모든 크롤링 완료
주소 검색 실패, 이름만 시도: 라마노피자
22 저장 완료 - 라마노피자
주소 검색 실패, 이름만 시도: 대왕사각피자
23 저장 완료 - 대왕사각피자
마지막 페이지
✅ 모든 크롤링 완료
24 저장 완료 - 비스트로위
25 저장 완료 - 오시게리따
마지막 페이지
✅ 모든 크롤링 완료
마지막 페이지
✅ 모든 크롤링

In [ ]:
browser.close()